In [1]:
import pandas as pd

In [2]:
med_new_wind = './medicanes_new_windows.csv'
df = pd.read_csv(med_new_wind)

In [4]:
mapping = df.groupby('id_final', as_index=False).agg(start_time=('start_time','first'),
                                                     end_time=('end_time','first')).sort_values('id_final')
mapping['start_time'] = pd.to_datetime(mapping['start_time'])
mapping['end_time'] = pd.to_datetime(mapping['end_time'])
mapping = mapping.sort_values('start_time').reset_index(drop=True)
mapping['durata'] = (mapping['end_time'] - mapping['start_time']).dt.total_seconds() / 3600
mapping['durata'] = mapping['durata'].round(1)
mapping

,id_final,start_time,end_time,durata
0,7001283,2010-10-12 12:55:00,2010-10-13 13:25:00,24.5
1,7001328,2011-11-05 22:55:00,2011-11-08 08:05:00,57.2
2,7001358,2012-04-13 04:15:00,2012-04-14 21:00:00,40.8
3,7001421,2013-11-18 22:00:00,2013-11-20 09:00:00,35.0
4,7001461,2014-11-07 00:10:00,2014-11-08 14:00:00,37.8
5,7001466,2014-11-30 06:25:00,2014-12-03 06:30:00,72.1
6,7001500,2015-09-30 21:25:00,2015-10-01 21:00:00,23.6
7,7001521,2016-02-28 23:20:00,2016-03-01 08:05:00,32.8
8,7001542,2016-10-28 06:30:00,2016-10-31 01:50:00,67.3
9,7001575,2017-11-16 14:45:00,2017-11-18 12:40:00,45.9


In [8]:
# costruisce la tabella a partire dai manifest che contengono solo 'datetime' (senza usare id_final)
from pathlib import Path
root = Path('../manifests')
manifests = [root/'./train.csv', root/'./val.csv', root/'./test.csv']
found = [p for p in manifests if p.exists()]
if not found:
    raise FileNotFoundError(f"Nessun manifest trovato tra: {', '.join(str(p) for p in manifests)}")

# leggi ogni manifest e annota la sorgente
dfs = []
for p in found:
    tmp = pd.read_csv(p)
    tmp['__source'] = p.name
    dfs.append(tmp)
combined = pd.concat(dfs, ignore_index=True)

if 'datetime' not in combined.columns:
    raise ValueError("I file manifest devono contenere la colonna 'datetime'.")

combined['datetime'] = pd.to_datetime(combined['datetime'])
combined = combined.sort_values('datetime').reset_index(drop=True)

# soglia per separare periodi (modifica se necessario)
threshold = pd.Timedelta('15min')

# crea segmenti quando il gap è maggiore della soglia
diff = combined['datetime'].diff().fillna(pd.Timedelta(seconds=0))
segments = (diff > threshold).cumsum()
combined['segment'] = segments

# aggrega per segmento per ottenere start/end, durata e i file sorgente coinvolti
periods = combined.groupby('segment', group_keys=False).agg(
    start_time=('datetime', 'min'),
    end_time=('datetime', 'max'),
    source_files=('__source', lambda s: ','.join(sorted(set(s))))
).reset_index(drop=True)

periods['durata'] = ((periods['end_time'] - periods['start_time']).dt.total_seconds() / 3600).round(1)

# mostra le colonne come nelle tabelle precedenti
periods = periods[['start_time', 'end_time', 'durata', 'source_files']]
periods

,start_time,end_time,durata,source_files
0,2010-10-11 00:00:00,2010-10-13 23:55:00,71.9,train.csv
1,2011-11-01 22:55:00,2011-11-12 08:05:00,249.2,train.csv
2,2012-04-09 04:15:00,2012-04-18 21:00:00,232.8,train.csv
3,2013-11-14 22:00:00,2013-11-24 09:00:00,227.0,train.csv
4,2014-11-03 00:10:00,2014-11-12 14:00:00,229.8,train.csv
5,2014-11-26 06:25:00,2014-12-07 06:30:00,264.1,train.csv
6,2015-09-26 21:25:00,2015-10-05 21:00:00,215.6,train.csv
7,2016-02-24 23:20:00,2016-03-05 08:05:00,224.8,train.csv
8,2016-10-24 06:30:00,2016-11-04 01:50:00,259.3,train.csv
9,2017-11-12 14:45:00,2017-11-21 09:00:00,210.2,train.csv


In [6]:
def merge_mapping_with_periods(mapping_med, periods_df, manifest_name=None):
    rows = []
    for _, m in mapping_med.iterrows():
        mask = (periods_df['start_time'] <= m['end_time_medicanes']) & (periods_df['end_time'] >= m['start_time_medicanes'])
        overlaps = periods_df[mask]
        if overlaps.empty:
            rows.append({
                'id_cyc': m.get('id_final', pd.NA),
                'start_time_medicanes': m['start_time_medicanes'],
                'end_time_medicanes': m['end_time_medicanes'],
                'durata_medicanes': m['durata_medicanes'],
                'start_time_manifest': pd.NaT,
                'end_time_manifest': pd.NaT,
                'durata_manifest': pd.NA,
                'manifest': manifest_name if manifest_name else pd.NaT
            })
        else:
            for _, o in overlaps.iterrows():
                rows.append({
                    'id_cyc': m.get('id_final', pd.NA),
                    'start_time_medicanes': m['start_time_medicanes'],
                    'end_time_medicanes': m['end_time_medicanes'],
                    'durata_medicanes': m['durata_medicanes'],
                    'start_time_manifest': o['start_time'],
                    'end_time_manifest': o['end_time'],
                    'durata_manifest': o['durata'],
                    'manifest': manifest_name if manifest_name else o['source_files']
                })
    return pd.DataFrame(rows)

### unisci mapping (medicanes) con le tabelle prodotte dai manifest per confrontare start/end

In [9]:
# prepara mapping_med con suffisso _medicanes
mapping_med = mapping.rename(columns={
    'start_time': 'start_time_medicanes',
    'end_time': 'end_time_medicanes',
    'durata': 'durata_medicanes'
})


# genera e mostra le tabelle unite per ogni manifest disponibile

merged = merge_mapping_with_periods(mapping_med, periods)
# filtra le righe con NaT
#merged = merged[merged['start_time_manifest'].notna()]
#merged_results[f'merged_{name}'] = merged
display(merged)

,id_cyc,start_time_medicanes,end_time_medicanes,durata_medicanes,start_time_manifest,end_time_manifest,durata_manifest,manifest
0,7001283,2010-10-12 12:55:00,2010-10-13 13:25:00,24.5,2010-10-11 00:00:00,2010-10-13 23:55:00,71.9,train.csv
1,7001328,2011-11-05 22:55:00,2011-11-08 08:05:00,57.2,2011-11-01 22:55:00,2011-11-12 08:05:00,249.2,train.csv
2,7001358,2012-04-13 04:15:00,2012-04-14 21:00:00,40.8,2012-04-09 04:15:00,2012-04-18 21:00:00,232.8,train.csv
3,7001421,2013-11-18 22:00:00,2013-11-20 09:00:00,35.0,2013-11-14 22:00:00,2013-11-24 09:00:00,227.0,train.csv
4,7001461,2014-11-07 00:10:00,2014-11-08 14:00:00,37.8,2014-11-03 00:10:00,2014-11-12 14:00:00,229.8,train.csv
5,7001466,2014-11-30 06:25:00,2014-12-03 06:30:00,72.1,2014-11-26 06:25:00,2014-12-07 06:30:00,264.1,train.csv
6,7001500,2015-09-30 21:25:00,2015-10-01 21:00:00,23.6,2015-09-26 21:25:00,2015-10-05 21:00:00,215.6,train.csv
7,7001521,2016-02-28 23:20:00,2016-03-01 08:05:00,32.8,2016-02-24 23:20:00,2016-03-05 08:05:00,224.8,train.csv
8,7001542,2016-10-28 06:30:00,2016-10-31 01:50:00,67.3,2016-10-24 06:30:00,2016-11-04 01:50:00,259.3,train.csv
9,7001575,2017-11-16 14:45:00,2017-11-18 12:40:00,45.9,2017-11-12 14:45:00,2017-11-21 09:00:00,210.2,train.csv


### per ogni manifest crea una tabella periodi separata (start_time, end_time, durata)

In [12]:
# per ogni manifest crea una tabella periodi separata (start_time, end_time, durata)
from pathlib import Path
root = Path('../manifests')
manifests = [root/'./train.csv', root/'./val.csv', root/'./test.csv']
found = [p for p in manifests if p.exists()]
if not found:
    raise FileNotFoundError(f"Nessun manifest trovato tra: {', '.join(str(p) for p in manifests)}")

from IPython.display import display

threshold = pd.Timedelta('15min')

results = {}
for p in found:
    dfp = pd.read_csv(p)
    if 'datetime' not in dfp.columns:
        raise ValueError(f"File {p.name} non contiene la colonna 'datetime'.")
    dfp['datetime'] = pd.to_datetime(dfp['datetime'])
    dfp = dfp.sort_values('datetime').reset_index(drop=True)
    diff = dfp['datetime'].diff().fillna(pd.Timedelta(seconds=0))
    segments = (diff > threshold).cumsum()
    periods = dfp.groupby(segments, group_keys=False).agg(
        start_time=('datetime', 'min'),
        end_time=('datetime', 'max')
    ).reset_index(drop=True)
    periods['durata'] = ((periods['end_time'] - periods['start_time']).dt.total_seconds() / 3600).round(1)
    periods = periods[['start_time', 'end_time', 'durata']]
    varname = f"periods_{p.stem}"
    results[varname] = periods

# mostra separatamente le tre tabelle (se esistono)
for name in ['periods_train', 'periods_val', 'periods_test']:
    if name in results:
        print(name)
        display(results[name])

periods_train


,start_time,end_time,durata
0,2011-11-01 22:55:00,2011-11-11 00:00:00,217.1
1,2014-11-03 00:10:00,2014-11-12 14:00:00,229.8
2,2016-10-26 00:00:00,2016-11-03 00:00:00,192.0
3,2017-11-13 00:00:00,2017-11-21 00:00:00,192.0
4,2019-11-06 09:30:00,2019-11-11 13:55:00,124.4
5,2019-11-11 14:35:00,2019-11-15 17:30:00,98.9
6,2020-09-10 23:50:00,2020-09-22 19:40:00,283.8


periods_val


,start_time,end_time,durata
0,2021-11-04 10:20:00,2021-11-09 09:00:00,118.7
1,2023-02-23 19:00:00,2023-03-06 13:30:00,258.5
2,2023-09-01 03:20:00,2023-09-12 09:15:00,269.9


periods_test


,start_time,end_time,durata
0,2020-12-09 08:20:00,2020-12-11 00:15:00,39.9
1,2020-12-11 01:15:00,2020-12-20 13:45:00,228.5
2,2020-12-21 10:35:00,2020-12-31 10:50:00,240.2
3,2021-10-29 00:00:00,2021-11-02 00:00:00,96.0


In [ ]:
# unisci mapping (medicanes) con le tabelle prodotte dai manifest per confrontare start/end
# prepara mapping_med con suffisso _medicanes
mapping['start_time'] = pd.to_datetime(mapping['start_time'])
mapping['end_time'] = pd.to_datetime(mapping['end_time'])
mapping['durata'] = mapping['durata'].round(1)
mapping_med = mapping.rename(columns={
    'start_time': 'start_time_medicanes',
    'end_time': 'end_time_medicanes',
    'durata': 'durata_medicanes'
})





# genera e mostra le tabelle unite per ogni manifest disponibile
merged_results = {}
for name in ['periods_train', 'periods_val', 'periods_test']:
    if name in results:
        merged = merge_mapping_with_periods(mapping_med, results[name], name.replace('periods_',''))
        # filtra le righe con NaT
        merged = merged[merged['start_time_manifest'].notna()]
        merged_results[f'merged_{name}'] = merged
        print(f"merged_{name}")
        display(merged)

merged_periods_train


,start_time_medicanes,end_time_medicanes,durata_medicanes,start_time_manifest,end_time_manifest,durata_manifest,manifest
1,2011-11-05 22:55:00,2011-11-08 08:05:00,57.2,2011-11-01 22:55:00,2011-11-11 00:00:00,217.1,train
4,2014-11-07 00:10:00,2014-11-08 14:00:00,37.8,2014-11-03 00:10:00,2014-11-12 14:00:00,229.8,train
8,2016-10-28 06:30:00,2016-10-31 01:50:00,67.3,2016-10-26 00:00:00,2016-11-03 00:00:00,192.0,train
9,2017-11-16 14:45:00,2017-11-18 12:40:00,45.9,2017-11-13 00:00:00,2017-11-21 00:00:00,192.0,train
10,2019-11-10 09:30:00,2019-11-11 17:30:00,32.0,2019-11-06 09:30:00,2019-11-11 13:55:00,124.4,train
11,2019-11-10 09:30:00,2019-11-11 17:30:00,32.0,2019-11-11 14:35:00,2019-11-15 17:30:00,98.9,train
12,2020-09-14 23:50:00,2020-09-18 19:40:00,91.8,2020-09-10 23:50:00,2020-09-22 19:40:00,283.8,train


merged_periods_val


,start_time_medicanes,end_time_medicanes,durata_medicanes,start_time_manifest,end_time_manifest,durata_manifest,manifest
15,2021-11-08 10:20:00,2021-11-09 06:35:00,20.2,2021-11-04 10:20:00,2021-11-09 09:00:00,118.7,val
16,2023-02-27 19:00:00,2023-03-02 13:30:00,66.5,2023-02-23 19:00:00,2023-03-06 13:30:00,258.5,val
17,2023-09-05 03:20:00,2023-09-10 02:00:00,118.7,2023-09-01 03:20:00,2023-09-12 09:15:00,269.9,val


merged_periods_test


,start_time_medicanes,end_time_medicanes,durata_medicanes,start_time_manifest,end_time_manifest,durata_manifest,manifest
12,2020-12-13 08:20:00,2020-12-16 13:45:00,77.4,2020-12-11 01:15:00,2020-12-20 13:45:00,228.5,test
13,2020-12-25 10:35:00,2020-12-27 10:50:00,48.2,2020-12-21 10:35:00,2020-12-31 10:50:00,240.2,test
14,2021-10-28 10:35:00,2021-10-30 13:00:00,50.4,2021-10-29 00:00:00,2021-11-02 00:00:00,96.0,test
